In [0]:
path = 'gdrive/My Drive/data/rdyn_data_6/results/2000_100_15_0.6_0.8_0.2_1_2.5_1/interactions.txt'
olcpm = 'gdrive/My Drive/data/olcpm/OLCPM/OLCPM.jar'

In [1]:
datapath = 'gdrive/My Drive/data/'
data_folder = 'rdyn_data_6/results/'
output_folder = datapath+'olcpm/'
algorithm_name = 'olcpm'
!pip install nf1

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

In [0]:
import pandas as pd

data = pd.read_csv(path, sep="\t", header=None)
data.columns = ["iteration", "timestamp", "action", "node_id1", "node_id2"]
data['node_id1'] = data['node_id1'].astype(str)
data["placeholder"] = " "
data['node_id2'] = data['node_id2'].astype(str)
data['edge'] = data[['node_id1','placeholder', 'node_id2']].apply(lambda x: ''.join(x), axis=1)
#data['edge2'] = data[['node_id2','placeholder', 'node_id1']].apply(lambda x: ''.join(x), axis=1)
data.head(n=10)



In [4]:
datapath = "gdrive/My Drive/data/rdyn_data_6"
data_folder= "/results/"
datapathoutput = "gdrive/My Drive/data/olcpm"
algorithm_name = 'olcpm'

#idc = data + '1000_100_15_0.6_0.8_0.2_1_3.5_90'
#gtc = data + '1000_100_15_0.6_0.8_0.2_1_3.5_90 2'

#rdyn_instance= "2000_100_15_0.6_0.8_0.2_1_2.5_0"
import os
import time
import pandas as pd
from nf1 import NF1
#results_fn = listdir van rdyn_instances


res_fn = os.listdir(datapath+data_folder)

print (res_fn)


def read_coms(filename):
        com = []
        with open(filename) as f:
            for l in f:
                try:
                    l = l.replace("(","[").replace(")","]").replace("]", "").replace(" ", "").split("[")[1]
                except:
                    pass
                ns = tuple(map(int, l.rstrip().replace(" ", "\t").replace(",", "\t").split("\t")))

                if len(ns) == 2:
                    raise Exception
                com.append(ns)
        return com

      
      
      
rows_list= []
for rdyn_instance in res_fn:  
    
    # get gt comms
    fn = os.listdir(datapath+data_folder+rdyn_instance)
    
    comms = [x for x in fn if 'communities' in x] 
    #print(comms)
    print(rdyn_instance, len(comms))
    #print(comms)
    
    
    comm_nr = []
    for c in comms:
        comm_nr.append(int(c[12:].replace(".txt",''),))
    #print (sorted(comm_nr))
    # get identified comms
    i_comms = os.listdir(datapathoutput+"/"+rdyn_instance+"/output_olcpm.txt")#output_folder + rdyn_instance)
    #print (i_comms)
    id_comms = [x for x in i_comms if "AlvCom" in x]

    
  
    gt_index = [[int(x[10:-4]),x] for x in id_comms]
    
 
    #print ("gt_index",gt_index)
    #print("id_comms", id_comms)
    
    
    count = 0
    # voor elke gt community
    for nr in comm_nr:
      #print("gt community nr:" , nr)
      # overloop alle 
      for el in gt_index:
        if nr == el[0]:
          #print (nr, el,)
          
          
          #print (idc,v)
          with open(datapathoutput+"/"+rdyn_instance+"/output_olcpm.txt/"+el[1])as f:# output_folder + rdyn_instance + "/"+v) as f:
              identified_coms = []
              com = []
              for l in f:
                  if l[0] =='#':
                    continue
                  #print("l",l)
                  #line
                  #line[2]= l.replace("(","[").replace(")","]").replace("]", "").replace(" ", "").split("[")[1]
                  line = l.split('\t')
                  #print ("1:", line)
                  line = line[1:]
                  #print(line[0], nr)
                  #print ("2:", line)
                  line[1] = line[1].replace("(","[").replace(")","]").replace("]", "").replace(" ", "").split("[")[1]
                  #print ("3:", line)
                  line[1] = [int(y) for y in line[1].replace("\n","").split(",")]
                  #print ("4:", s)
                  #print ("5:", line)
                  #print ("x",)
                  #print("qdfqfdsf",el[0], line[0], nr)
                  #print("current:", el[0], line[0])
                  if str(el[0]) == line[0]:
                      identified_coms.append(tuple(line[1]))
              #break        
              #print(identified_coms)
              gt = read_coms(datapath+data_folder+rdyn_instance+"/communities-"+str(el[0])+".txt")
              #print(gt)
              #break
              #count +=1
              #print(c,datapath+data_folder+rdyn_instance+'/'+comms[c])






              if identified_coms == []:
                      continue
              #print(output_folder + rdyn_instance + "/"+v)
              #print(comm_nr)
              #print(comms)





              nf = NF1(identified_coms, gt)
              #print(nf.summary())
              res = []
              res.extend(nf.get_f1())
              #print(nf.get_partition_stats())
              #print ()
              mean = str(res[0])
              std = str(res[1])
              maxx = str(res[2])
              minn = str(res[3])
              mode =str(res[4])


              row = {
                  "algorithm": algorithm_name,
                  "rdyn_instance": str(rdyn_instance),
                  #"graph_n": str(alg_results_fn[i]),
                  "mean": mean.replace(".",","),
                  "std" : std.replace(".",","),
                  "min": minn.replace(".",","),
                  "max": maxx.replace(".",","),
                  "mode": mode.replace(".",",")
              }
              rows_list.append(row)
      #print(rows_list)
      #print(os.getcwd())
      #else:
          #print('skip')

df = pd.DataFrame(rows_list)  
#print(datapath+"output_4000_"+algorithm_name+".txt")
df.to_csv(datapath+"output_1000_t_"+algorithm_name+".txt", sep=' ', index=False)


['2000_100_15_0.6_0.8_0.2_1_2.5_0', '2000_100_15_0.6_0.8_0.2_1_3_0', '2000_100_15_0.6_0.8_0.2_1_3.5_0', '2000_100_15_0.6_0.8_0.2_1_2.5_1', '2000_100_15_0.6_0.8_0.2_1_3_1', '2000_100_15_0.6_0.8_0.2_1_3.5_1', '2000_100_15_0.6_0.8_0.2_1_2.5_2', '2000_100_15_0.6_0.8_0.2_1_3_2', '2000_100_15_0.6_0.8_0.2_1_3.5_2', '2000_100_15_0.6_0.8_0.2_1_2.5_3', '2000_100_15_0.6_0.8_0.2_1_3_3', '2000_100_15_0.6_0.8_0.2_1_3.5_3', '2000_100_15_0.6_0.8_0.2_1_2.5_4', '2000_100_15_0.6_0.8_0.2_1_3_4', '2000_100_15_0.6_0.8_0.2_1_3.5_4', '2000_100_15_0.6_0.8_0.2_1_2.5_5', '2000_100_15_0.6_0.8_0.2_1_3_5', '2000_100_15_0.6_0.8_0.2_1_3.5_5', '2000_100_15_0.6_0.8_0.2_1_2.5_6', '2000_100_15_0.6_0.8_0.2_1_3_6', '2000_100_15_0.6_0.8_0.2_1_3.5_6', '2000_100_15_0.6_0.8_0.2_1_2.5_7', '2000_100_15_0.6_0.8_0.2_1_3_7', '2000_100_15_0.6_0.8_0.2_1_3.5_7', '2000_100_15_0.6_0.8_0.2_1_2.5_8', '2000_100_15_0.6_0.8_0.2_1_3_8', '2000_100_15_0.6_0.8_0.2_1_3.5_8', '2000_100_15_0.6_0.8_0.2_1_2.5_9', '2000_100_15_0.6_0.8_0.2_1_3_9', '200

In [0]:
i = "/interactions.txt"
import pandas as pd
import os
import shutil
import time
os.chdir("/content")
#print(os.getcwd())

results_fn = os.listdir(datapath+data_folder)
print(len(results_fn))
times= []

for _, rdyn_instance in enumerate(results_fn):
    print(_, rdyn_instance)
    data = pd.read_csv(datapath+data_folder+rdyn_instance+i, sep="\t", header=None)
    data.columns = ["iteration", "timestamp", "action", "node_id1", "node_id2"]
    data['node_id1'] = data['node_id1'].astype(str)
    data["placeholder"] = " "
    data['node_id2'] = data['node_id2'].astype(str)
    data['edge'] = data[['node_id1','placeholder', 'node_id2']].apply(lambda x: ''.join(x), axis=1)
    #data['edge2'] = data[['node_id2','placeholder', 'node_id1']].apply(lambda x: ''.join(x), axis=1)
    data.head(n=10)


    
           
    os.mkdir(datapath + 'olcpm/'+rdyn_instance)
    src = 'gdrive/My Drive/data/olcpm/OLCPM.jar'
    dst = datapath + 'olcpm/'+rdyn_instance + '/OLCPM.jar'
    shutil.copy(src, dst)
    
    
    nodes = set()
    edges = set()
    edges_s = []
    day = 1
    res = []


    for index, row in data.iterrows():
        if row["node_id1"] not in nodes:
            nodes.add(row["node_id1"])
            res.append(["+n\t" + str(row["node_id1"]) + "\t"+ str(row["iteration"]*day)])
        if row["node_id2"] not in nodes:
            nodes.add(row["node_id2"])
            res.append(["+n\t" + str(row["node_id2"]) + "\t"+ str(row["iteration"]*day)])
        if row["edge"] not in edges:
            if row["node_id1"] < row["node_id2"]:
                res.append([row['action']+"l"+"\t" + str(row["node_id1"]) + "\t" + str(row["node_id2"]) + "\t" + str(row["iteration"]*day)])
            else:
                res.append([row['action']+"l"+"\t" + str(row["node_id2"]) + "\t" + str(row["node_id1"]) + "\t" + str(row["iteration"]*day)])
        edges_s.append(str(row['action']) + str(row['edge']))
    
    
    with open(output_folder+rdyn_instance+"/input_olcpm.txt", 'w') as f:

        for r in res:
            f.write("{0}\n".format(r[0]))


     # At this point the necessary files for one rdyn instance are made.
    
    import os

     # move jar in map and run algorithm.
    
    os.chdir(datapath + '/olcpm/'+rdyn_instance)
    print(os.getcwd())
    start = time.time()
    !java -jar OLCPM.jar -i input_olcpm.txt -o output_olcpm.txt > /dev/null
    end = time.time()
    x = str(end-start)
        
    run_time = {   "algorithm": algorithm_name,
                   "rdyn_instance_name" : rdyn_instance,
                   "execution_time" : x.replace(".",",") 
    }
    times.append(run_time)
    os.chdir("/content")
    #content/gdrive/My Drive/data/rdyn_data/results_4000
    
df = pd.DataFrame(times)  
df.to_csv(datapath+"execution_times_1000__"+algorithm_name+".txt", sep=' ', index=False)

In [0]:
nodes = set()
edges = set()
edges_s = []
day = 86400
res = []


for index, row in data.iterrows():
    if row["node_id1"] not in nodes:
        nodes.add(row["node_id1"])
        res.append(["+n\t" + str(row["node_id1"]) + "\t"+ str(row["iteration"]*day)])
    if row["node_id2"] not in nodes:
        nodes.add(row["node_id2"])
        res.append(["+n\t" + str(row["node_id2"]) + "\t"+ str(row["iteration"]*day)])
    if row["edge"] not in edges:
        if row["node_id1"] < row["node_id2"]:
            res.append([row['action']+"l"+"\t" + str(row["node_id1"]) + "\t" + str(row["node_id2"]) + "\t" + str(row["iteration"]*day)])
        else:
            res.append([row['action']+"l"+"\t" + str(row["node_id2"]) + "\t" + str(row["node_id1"]) + "\t" + str(row["iteration"]*day)])
    edges_s.append(str(row['action']) + str(row['edge']))
    
    
print (len(res))
with open("gdrive/My Drive/data/olcpm/OLCPM/input_olcpm.txt", 'w') as f:
    
    for r in res:
        f.write("{0}\n".format(r[0]))

        

        
        
import os
os.chdir('gdrive/My Drive/data/olcpm/OLCPM')
!java -jar OLCPM.jar -i input_olcpm.txt -o output_olcpm.txt > /dev/null
os.chdir('/content')


In [0]:
print (len('AlvComStep58'))

12
